In [2]:
import pandas as pd

def ichimoku_cloud(
    data: pd.DataFrame,
    small_period: int = 7,
    large_period: int = 42,
    rolling_period: int = 60,
):

    small_period_high = data["HA_HIGH"].rolling(window=small_period).max()
    small_period_low = data["HA_LOW"].rolling(window=small_period).min()
    data["tenkan_sen"] = (small_period_high + small_period_low) / 2

    large_period_high = data["HA_HIGH"].rolling(window=large_period).max()
    large_period_low = data["HA_LOW"].rolling(window=large_period).min()
    data["kijun_sen"] = (large_period_high + large_period_low) / 2

    data["senkou_span_a"] = (data["tenkan_sen"] + data["kijun_sen"]) / 2
    data["senkou_span_a"] = data["senkou_span_a"].shift(20)
    data["senkou_span_b"] = (
        data["HA_HIGH"].rolling(window=rolling_period).max()
        + data["HA_LOW"].rolling(window=rolling_period).min()
    ) / 2
    data["senkou_span_b"] = data["senkou_span_b"].shift(40)
    cloud_upper_bound = []
    cloud_lower_bound = []
    cloud_width = []
    data['cloud_threshold'] = 0
    for i in range(len(data)):

        cloud_width.append(abs(data['senkou_span_a'][i] - data['senkou_span_b'][i]))
        cloud_upper_bound.append(max(data['senkou_span_a'][i],data['senkou_span_b'][i]))
        cloud_lower_bound.append(min(data['senkou_span_a'][i],data['senkou_span_b'][i]))
        data['cloud_threshold'][i] = (cloud_upper_bound[-1] - cloud_lower_bound[-1])
    cloud_threshold = data['cloud_threshold'].rolling(window=11).mean()
    return data

In [ ]:
def signals_ichimoku(data):

    cloud_upper_bound = []
    cloud_lower_bound = []
    cloud_width = []
    signals_ich_1 = []
    signals_ich_2 = []
    signals_ich_3 = []
    data['cloud_threshold'] = 0
    for i in range(len(data)):

        senkou_a = data['senkou_span_a'][i]
        senkou_b = data['senkou_span_b'][i]
        cloud_width_value = abs(senkou_a - senkou_b)
        cloud_upper = max(senkou_a, senkou_b)
        cloud_lower = min(senkou_a, senkou_b)
        
        cloud_width.append(cloud_width_value)
        cloud_upper_bound.append(cloud_upper)
        cloud_lower_bound.append(cloud_lower)
        
        data.at[i, 'cloud_threshold'] = cloud_upper - cloud_lower
        if i >= 10:
            cloud_threshold = data['cloud_threshold'].iloc[i-10:i+1].mean()
            if data['close'][i] > cloud_upper and cloud_width_value > 2 * cloud_threshold:
                signals_ich_1.append(1)
            elif data['close'][i] < cloud_lower and cloud_width_value > 2 * cloud_threshold:
                signals_ich_1.append(-1)
            else:
                signals_ich_1.append(0)
        else:
            signals_ich_1.append(0)
        

        if i > 0: 
            if (data['tenkan_sen'][i] > data['kijun_sen'][i] and 
                data['tenkan_sen'][i-1] < data['kijun_sen'][i-1]):
                signals_ich_2.append(1)
            elif (data['tenkan_sen'][i] < data['kijun_sen'][i] and 
                  data['tenkan_sen'][i-1] > data['kijun_sen'][i-1]):
                signals_ich_2.append(-1)
            else:
                signals_ich_2.append(0)
        else:
            signals_ich_2.append(0)
        if data['HA_CLOSE'][i] > senkou_a and senkou_a > senkou_b:
            signals_ich_3.append(1)
        elif data['HA_CLOSE'][i] < senkou_a and senkou_a < senkou_b:
            signals_ich_3.append(-1)
        else:
            signals_ich_3.append(0)
    
    data['signals_ich_1'] = signals_ich_1
    data['signals_ich_2'] = signals_ich_2
    data['signals_ich_3'] = signals_ich_3

    return data
